In [1]:
import sounddevice as sd
import numpy as np

In [2]:
FS = 44100 #sampling frequency

In [3]:
def create_t(length_seconds):
    return np.linspace(0, length_seconds, FS*length_seconds)

In [4]:
S = np.sin(create_t(1)*2*np.pi* 200) #sinusoid at 200Hz